# Connect to the database

Database connection is created through create_engine. The string provides the database connector, username, password, database server:port and the database name. 

In [1]:
from sqlalchemy import create_engine, text, inspect, Table
import pandas as pd

engine = create_engine("postgresql://student:infomdss@db_dashboard:5432/dashboard")

with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS pets CASCADE;"))
    conn.execute(text("DROP TABLE IF EXISTS owners CASCADE;"))
    conn.execute(text("DROP TABLE IF EXISTS procedures CASCADE;"))

# Load the CSV files

We will read the files, verify that they have been read, and later dump them to the database

In [2]:
pets_df = pd.read_csv("Pets.csv", delimiter=",")
owners_df = pd.read_csv("Owners.csv", delimiter=",")
procedures_df = pd.read_csv("ProceduresHistory.csv", delimiter=",")
print (pets_df.head())

     petid     name    kind  gender  age  ownerid
0  J6-8562  Blackie     Dog    male   11     5168
1  Q0-2001   Roomba     Cat    male    9     5508
2  M0-2904    Simba     Cat    male    1     3086
3  R3-7551   Keller  Parrot  female    2     7908
4  P2-7342  Cuddles     Dog    male   13     4378


In [3]:
pets_df.to_sql("pets", engine, if_exists="replace", index=True)
owners_df.to_sql("owners", engine, if_exists="replace", index=True)
procedures_df.to_sql("procedures", engine, if_exists="replace", index=True)

284

In the following block, we read the table from the database as a dataframe. 

In [4]:
procedures_table = pd.read_sql_table('procedures', engine, index_col='index')
procedures_table

,petid,date,proceduretype,proceduresubcode
index,,,,
0,A8-1181,2016-01-10,VACCINATIONS,5
1,E7-3766,2016-01-11,VACCINATIONS,5
2,B8-8740,2016-01-11,VACCINATIONS,5
3,D4-9443,2016-01-11,VACCINATIONS,5
4,F6-3398,2016-01-12,HOSPITALIZATION,1
...,...,...,...,...
2279,A4-2573,2016-12-30,VACCINATIONS,5
2280,B3-8317,2016-12-30,VACCINATIONS,5
2281,C5-4291,2016-12-30,ORTHOPEDIC,3


Following block answers one of the questions where you have to show the information of names of the owners and procedures they had for their pets. We have three tables and we need to join them together to get the combined information.

In [5]:
query_1 = '''SELECT o.name, o.surname, p.proceduretype 
 FROM procedures as p 
 INNER JOIN Pets as a
ON p.petid = a.petid
 INNER JOIN owners as o
 ON a.ownerid = o.ownerid
 ORDER BY o.name
 '''
con = engine.connect()
#############################
table_1 = pd.read_sql(query_1, con) if not query_1.isspace() else 0
table_1

,name,surname,proceduretype
0,Arthur,Reed,VACCINATIONS
1,Bruce,Dunne,VACCINATIONS
2,Bruce,Tatum,VACCINATIONS
3,Bruce,Dunne,VACCINATIONS
4,Carolyn,Crane,VACCINATIONS
5,Charles,Swarey,GROOMING
6,Christopher,Bowers,ORTHOPEDIC
7,Dan,Hysell,VACCINATIONS
8,Daniel,Fay,GENERAL SURGERIES
9,Doris,Ray,GROOMING


In [6]:
query_4 = '''SELECT o.name, COUNT(*) as Total
FROM pets as a
LEFT OUTER JOIN owners as o
ON a.ownerid = o.ownerid
GROUP BY o.name
ORDER BY Total DESC
'''
con = engine.connect()
#############################
table_4 = pd.read_sql(query_4, con) if not query_4.isspace() else 0
table_4

,name,total
0,Robert,6
1,Charles,4
2,Bruce,3
3,Susan,3
4,John,3
...,...,...
70,Hassan,1
71,Mary,1
72,Enrique,1
73,Dan,1
